In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import random
from sklearn import preprocessing
from keras.preprocessing.image import img_to_array

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#! tar -xvf 'dataset.tar.xz'

In [ ]:
# defining global variable path
image_path = "/content/dataset"
label_path = "/content/dataset/train.csv"
img_size = 224

In [ ]:
batch_size = 64
epochs = 10

### Load the images and the labels (from path to array)

In [ ]:
def load_images_labels(image_path, label_path):
    image_files = sorted([os.path.join(image_path, 'train', file)
         for file in os.listdir(image_path + "/train") if file.endswith('.jpg')])
    random.seed(42)
    random.shuffle(image_files)
    
    labels_df = pd.read_csv(label_path)
    labels_dict = {}
    for pair in labels_df.iloc[0:,0:2].values:
        labels_dict[pair[0]] = pair[1]
        
    labels = []
    for img in image_files:
        labels.append(labels_dict[img.split('/')[-1].split('.')[0]])
    
    lb = preprocessing.LabelBinarizer()
    labels = lb.fit_transform(labels)
    classes = lb.classes_
    
    return image_files, labels, classes

In [ ]:
image_files, labels, classes = load_images_labels(image_path, label_path)
print("Total Images : ",len(image_files))
print("Total Labels : ",len(labels))
print("Total Classes : ",len(classes))

Total Images :  5508
Total Labels :  5508
Total Classes :  35


### Preprocessing and Data Augmentation

In [ ]:
'''
def display(a, b, title1 = "Original", title2 = "Edited"):
    plt.subplot(121), plt.imshow(a), plt.title(title1)
    plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(b), plt.title(title2)
    plt.xticks([]), plt.yticks([])
    plt.show()
    
def processing(data):
    img = [cv2.imread(i, cv2.IMREAD_UNCHANGED) for i in data]
    original = img[0]
    
    # Resizing the images
    height = 224
    width = 224
    dim = (width, height)
    res_img = []
    for i in range(len(img)):
        res = cv2.resize(img[i], dim, interpolation=cv2.INTER_LINEAR)
        res_img.append(res)
    print('Original size',img[0].shape)
    print("Resized Shapes", res_img[1].shape)
    display(original,res_img[0],'Original','Resized')
    
    # Denoising the images
    no_noise = []
    for i in range(len(res_img)):
        blur = cv2.GaussianBlur(res_img[i], (5, 5), 0)
        no_noise.append(blur)
    display(original, no_noise[0], 'Original', 'Blured')
    return no_noise
'''

'\ndef display(a, b, title1 = "Original", title2 = "Edited"):\n    plt.subplot(121), plt.imshow(a), plt.title(title1)\n    plt.xticks([]), plt.yticks([])\n    plt.subplot(122), plt.imshow(b), plt.title(title2)\n    plt.xticks([]), plt.yticks([])\n    plt.show()\n    \ndef processing(data):\n    img = [cv2.imread(i, cv2.IMREAD_UNCHANGED) for i in data]\n    original = img[0]\n    \n    # Resizing the images\n    height = 224\n    width = 224\n    dim = (width, height)\n    res_img = []\n    for i in range(len(img)):\n        res = cv2.resize(img[i], dim, interpolation=cv2.INTER_LINEAR)\n        res_img.append(res)\n    print(\'Original size\',img[0].shape)\n    print("Resized Shapes", res_img[1].shape)\n    display(original,res_img[0],\'Original\',\'Resized\')\n    \n    # Denoising the images\n    no_noise = []\n    for i in range(len(res_img)):\n        blur = cv2.GaussianBlur(res_img[i], (5, 5), 0)\n        no_noise.append(blur)\n    display(original, no_noise[0], \'Original\', \'Blu

In [ ]:
X = []
for img in image_files:
  i = cv2.imread(img)
  i = cv2.resize(i, (img_size, img_size))
  i = img_to_array(i)
  X.append(i)

In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
x_train, x_val, y_train, y_val = train_test_split(X, labels, test_size=0.20)
x_train = np.array(x_train)/255
x_val = np.array(x_val) 
y_train = np.array(y_train)
y_val = np.array(y_val)

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(
    x_train,
    y=y_train,
    batch_size=batch_size,
    shuffle=True,
    sample_weight=None,
    seed=None,
    save_to_dir=None,
    subset=None)

validation_generator = validation_datagen.flow(
    x_val,
    y=y_val,
    batch_size=batch_size,
    shuffle=True,
    sample_weight=None,
    seed=None,
    save_to_dir=None,
    subset=None)

### Building the Model

[Conv -> Pool] -> [Conv -> Pool] -> [Conv] -> [Conv] -> [Conv -> Pool] -> Flatten -> Full Connection

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers

In [ ]:
# Initializing the model
model = Sequential()

In [ ]:
# Convolution Layer 
model.add(Convolution2D(96, 11, strides = (4, 4), padding = 'valid', input_shape=(224, 224, 3), activation = 'relu'))
# Max Pooling Layer 
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
# Batch Normalization 
model.add(BatchNormalization())

# Convolution Layer 
model.add(Convolution2D(256, 11, strides = (1, 1), padding='valid', activation = 'relu'))
# Max Pooling Layer 
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding='valid'))
# Batch Normalization
model.add(BatchNormalization())

# Convolution Layer 
model.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
# Batch Normalization
model.add(BatchNormalization())

# Convolution Layer 
model.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
# Batch Normalization
model.add(BatchNormalization())

# Convolution Layer 
model.add(Convolution2D(256, 3, strides=(1,1), padding='valid', activation = 'relu'))
# Max Pooling Layer 
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
# Batch Normalization
model.add(BatchNormalization())

# Flattening Step
model.add(Flatten())

# Full Connection Step
model.add(Dense(units = 4096, activation = 'relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(units = 4096, activation = 'relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(units = 1000, activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(units = 35, activation = 'softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 256)         1024      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 384)        

In [ ]:
model.compile(optimizer=optimizers.SGD(lr=0.001),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, steps_per_epoch = len(x_train) // batch_size,
    validation_data=validation_generator, validation_steps=len(x_val) // batch_size,
    epochs=epochs, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
68/68 [==============================] - 766s 11s/step - loss: 4.4017 - accuracy: 0.0306 - val_loss: 3.5262 - val_accuracy: 0.0285
Epoch 2/10
68/68 [==============================] - 763s 11s/step - loss: 4.3825 - accuracy: 0.0318 - val_loss: 3.6298 - val_accuracy: 0.0279
Epoch 3/10
68/68 [==============================] - 762s 11s/step - loss: 4.3434 - accuracy: 0.0329 - val_loss: 4.0391 - val_accuracy: 0.0385
Epoch 4/10
68/68 [==============================] - 763s 11s/step - loss: 4.3222 - accuracy: 0.0341 - val_loss: 7.3665 - val_accuracy: 0.0308
Epoch 5/10
 9/68 [==>...........................] - ETA: 10:33 - loss: 4.2337 - accuracy: 0.0486